In [3]:
'''This is a reproduction of the IRNN experiment
with pixel-by-pixel sequential MNIST in
"A Simple Way to Initialize Recurrent Networks of Rectified Linear Units"
by Quoc V. Le, Navdeep Jaitly, Geoffrey E. Hinton
arxiv:1504.00941v2 [cs.NE] 7 Apr 2015
http://arxiv.org/pdf/1504.00941v2.pdf
Optimizer is replaced with RMSprop which yields more stable and steady
improvement.
Reaches 0.93 train/test accuracy after 900 epochs
(which roughly corresponds to 1687500 steps in the original paper.)
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras import initializers
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [4]:
batch_size = 32
num_classes = 10
hidden_units = 100

learning_rate = 1e-6
clip_norm = 1.0

In [47]:
# the data, shuffled and split between train and test sets
(x_train_all, y_train_all), (x_test_all, y_test_all) = mnist.load_data()
print(x_train_all.shape,y_train_all.shape)

(60000, 28, 28) (60000,)


In [48]:
n_samples = 10000
n_valid_samples = 5000

x_train = x_train_all[:n_samples]
y_train = y_train_all[:n_samples]
x_test = x_test_all[:n_valid_samples]
y_test = y_test_all[:n_valid_samples]

In [49]:
x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (10000, 784, 1)
10000 train samples
5000 test samples


In [50]:
print(x_train.shape[1:])

(784, 1)


In [51]:
print('Evaluate IRNN...')
model = Sequential()
model.add(SimpleRNN(hidden_units,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.summary())

Evaluate IRNN...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 100)               10200     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 11,210
Trainable params: 11,210
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
epochs = 2
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 10000 samples, validate on 5000 samples
Epoch 1/2
10000/10000 [==============================] - 172s - loss: 2.2885 - acc: 0.1122 - val_loss: 2.2717 - val_acc: 0.1858

In [28]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('IRNN test score:', scores[0])
print('IRNN test accuracy:', scores[1])


IRNN test score: 2.2934889679
IRNN test accuracy: 0.0880000002384


In [46]:
W = model.layers[0].get_weights()
W[0].shape, W[1].shape,W[2].shape

(array([  0.00000000e+00,  -4.25768167e-06,  -1.64816374e-05,
         -7.47436934e-06,   0.00000000e+00,  -2.63533802e-05,
         -3.63329709e-05,   0.00000000e+00,   0.00000000e+00,
         -4.66929814e-05,  -4.05077080e-05,  -5.07591085e-06,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -3.37011406e-05,   2.50338326e-05,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,  -7.36012453e-06,
          0.00000000e+00,   1.76317008e-05,   2.21621140e-05,
         -2.06327040e-05,  -4.23146448e-05,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          3.01771088e-05,   2.46748605e-05,  -4.92670051e-06,
         -3.56911223e-05,   7.01268391e-06,  -2.82598171e-06,
          5.54729195e-05,  -4.60707497e-06,   0.00000000e+00,
         -3.94934686e-05,   4.40784152e-06,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -5.38437671e-05,   2.96559738e-05,   2.76190385e-05,
        

In [30]:
yhat = model.predict(x_test[0:1])

In [31]:
yhat

array([[ 0.11052388,  0.09168478,  0.10322027,  0.10358814,  0.10393184,
         0.09456956,  0.09753137,  0.10193805,  0.09363104,  0.09938099]], dtype=float32)